# Notebook for getting results of model

**Purpose of notebook is:**
1. Aggregate our predictions to one predction per file
  1. ~~Trying majority voting, softmax(sum(logits)), or greedy method~~
  2. ~~Running the model on the test dataset~~
  3. ~~Output file for leaderboard~~
  1. All done. Just creates a final prediction for the selected model

## Imports and setup

In [2]:
#%pip install -e 'src/hear21passt' 
%pip install -e 'src/hear21passt'

Obtaining file:///root/data/exploratory_notebooks/da/src/hear21passt
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 26.5 MB/s eta 0:00:00
  Running setup.py develop for hear21passt

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Hack: kill the process to restart the kernel and load packages
import os
os._exit(0)

### Imports

In [ ]:
import os

import torch
import pandas as pd
import tqdm

from torch.optim import AdamW
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from torch.nn import functional as F

from utils.model import get_pretrained_passt_model

from utils.gdsc22_dataset_legacy import BuzzMapTestDataset
from utils.dataset_augmentations import BuzzMapTransformedTestDataset, BuzzMapTransformedDataset, NormalizeAmplitudes

### Constants

In [5]:
# List which models we can currently selecting from
!ls -ahl ../../checkpoints/

total 72G
drwxr-xr-x  2 root root     22K Jul 16 16:16  .
drwx------ 21 root nogroup 6.0K Jul 16 13:52  ..
-rw-r--r--  1 root root    978M Jul 12 16:30  gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_epochs40_gain_roll_mixup_noise@2023-07-12T16:26:10.pt
-rw-r--r--  1 root root    978M Jul 12 17:16  gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_epochs40_gain_roll_mixup_noise@2023-07-12T17:01:22.pt
-rw-r--r--  1 root root    978M Jul 12 17:26  gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_epochs40_gain_roll_mixup_noise@2023-07-12T17:16:51.pt
-rw-r--r--  1 root root    978M Jul 12 22:09  gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_epochs40_gain_roll_mixup_noise@2023-07-12T17:28:35.pt
-rw-r--r--  1 root root    978M Jul 13 03:43  gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_epochs40_gain_roll_mixup_noise@2023-07-12T22:47:03.pt
-rw-r--r--  1 root root    978M Jul 14 20:10 'gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_fixed_shuffle_epochs20_transformation=[normed]_epochs20_transformations

In [2]:
# Choose hich model to test
MODEL_NAME = "gdsc22_passt_t40_f4.pt"
MODEL_NAME = 'gdsc22_passt_kd_NOEXP_lr_mismatch_size_t10_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed]@2023-07-16T16:44:23_epoch=9_epochs=[0, 10].pt'

# Helper for later - could be refactored
mname = MODEL_NAME[0:MODEL_NAME.rfind(".")]

In [3]:
# Even more constants
CLASSIFIER_N_CLASSES = 66
DEBUG_TRANSFORMATIONS = False

base_dir = os.path.abspath(os.path.join(os.getcwd(), '../../gdsc_data/data_processed/data_8c86715/06_resample_chunk_test_into_non_overlapping_10sec_window'))
test_dir = os.path.join(base_dir, 'test/')

In [4]:
# Generate dataset and dataloader
test_dataset = BuzzMapTestDataset(
    test_dir,
    metadata_csv=os.path.join(test_dir, 'metadata.csv'), 
    num_classes=CLASSIFIER_N_CLASSES
)

test_dataset_normed = BuzzMapTransformedTestDataset(
    transformers=[NormalizeAmplitudes(debug=DEBUG_TRANSFORMATIONS)],
    buzz_mappable=test_dataset,
    debug=True    
)

test_dataloader = DataLoader(test_dataset_normed, batch_size=30)

# Delete this cell below once it works
Only used to test if the dataloader iterates correctly, before the model prints too much

## Load the model

In [5]:
TORCH_SAVE_PATH = os.path.abspath(os.path.join(os.getcwd(), f'../../checkpoints/{MODEL_NAME}'))

#model_eval = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=0, s_patchout_f=0, debug=False)
model_eval = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=0, s_patchout_f=0)
state = torch.load(TORCH_SAVE_PATH)
model_eval.load_state_dict(state['model_state_dict'])



 Loading PaSST pre-trained on AudioSet (with KD) Patch 16 stride 10 structured patchout mAP=486 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
   

<All keys matched successfully>

In [6]:
model_eval = model_eval.cuda() if torch.cuda.is_available() else model_eval.cpu() 

In [10]:
next(iter(test_dataloader))

[('0.wav',
  '1.wav',
  '3.wav',
  '3.wav',
  '3.wav',
  '3.wav',
  '3.wav',
  '2.wav',
  '2.wav',
  '2.wav',
  '2.wav',
  '2.wav',
  '4.wav',
  '5.wav',
  '7.wav',
  '7.wav',
  '7.wav',
  '7.wav',
  '8.wav',
  '8.wav',
  '6.wav',
  '6.wav',
  '6.wav',
  '6.wav',
  '6.wav',
  '6.wav',
  '6.wav',
  '6.wav',
  '6.wav',
  '10.wav'),
 ('0_1.wav',
  '1_1.wav',
  '3_1.wav',
  '3_2.wav',
  '3_3.wav',
  '3_4.wav',
  '3_5.wav',
  '2_1.wav',
  '2_2.wav',
  '2_3.wav',
  '2_4.wav',
  '2_5.wav',
  '4_1.wav',
  '5_1.wav',
  '7_1.wav',
  '7_2.wav',
  '7_3.wav',
  '7_4.wav',
  '8_1.wav',
  '8_2.wav',
  '6_1.wav',
  '6_2.wav',
  '6_3.wav',
  '6_4.wav',
  '6_5.wav',
  '6_6.wav',
  '6_7.wav',
  '6_8.wav',
  '6_9.wav',
  '10_1.wav'),
 ("STEP1:<class 'utils.dataset_augmentations.NormalizeAmplitudes'>: Normalized amplitude '-28.5197811126709' by '8.519781112670898'. Target dBFS='-20'",
  "STEP1:<class 'utils.dataset_augmentations.NormalizeAmplitudes'>: Normalized amplitude '-42.245826721191406' by '22.24582

# Run predictions on test dataset

In [7]:
ypred_f1 = []
ytrue_f1 = []
fnames_test = []
name_split_list = []
probs_list_test = []

model_eval.eval()

cnt = 0
with torch.no_grad():
    
    #for fname, name_split, wav in tqdm.tqdm(test_dataloader):
    for fname, name_split, _, wav, _  in tqdm.tqdm(test_dataloader):
        cnt = cnt + 1
        #if cnt > 2:
        #    break
        
        # Store which files we are running on
        fnames_test.append(fname)
        name_split_list.append(name_split)
        wav = wav.cuda()
        
        # Evaluate
        y_hat, emb = model_eval(wav)
        
        # Get probabilities, and append
        prob = F.softmax(y_hat, dim=1)
        probs_list_test.append(prob.cpu())
        # Get prediction (but not really used for anything)
        y_pred = torch.argmax(prob, dim=1)        
        ypred_f1.append(y_pred.cpu())

  0%|          | 0/58 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([30, 1, 128, 1000])
self.norm(x) torch.Size([30, 768, 12, 99])
 patch_embed :  torch.Size([30, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X flattened torch.Size([30, 1188, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([30, 1, 768])
 self.dist_token.shape torch.Size([30, 1, 768])
 final sequence x torch.Size([30, 1190, 768])
 after 12 atten blocks x torch.Size([30, 1190, 768])
forward_features torch.Size([30, 768])
head torch.Size([30, 66])


100%|██████████| 58/58 [03:25<00:00,  3.55s/it]


# Gather results to dataframe

In [8]:
# Get the probabilities for each class
df_test = pd.DataFrame(torch.cat(probs_list_test).numpy(), columns = list(range(0,66,1)))

# Append filename
df_test['fname'] = [item for t in fnames_test for item in t]
df_test['name_split'] = [item for t in name_split_list for item in t]

# Reorder - Can be done a lot cleaner
cols = df_test.columns.tolist()
#cols = cols[-2:] + cols[:-2]
df_test = df_test[cols]


## Split the fname, so we know which files were chunked together
#def splitter(txt):
#    x = txt.rfind("_")
#    y = txt.rfind(".")
#    original_file = txt[0:x] + txt[y:len(txt)]
#    #chunk_no = int(txt[x+1:y])
#    chunk_no = txt[x+1:y] # Should be int, but broken because of reasons. Will be fixed when data_processed is recreated
#    return original_file, chunk_no

## Create helper
#df_test['tmp'] = df_test['fname'].apply(splitter)

## Create new columns
#new_col_list = ['original_filename', 'chunk_no']
#for n,col in enumerate(new_col_list):
#    df_test[col] = df_test['tmp'].apply(lambda tmp: tmp[n])

## Clean up my own mess
#df_test = df_test.drop('tmp',axis=1)

cols = df_test.columns.tolist()
cols = cols[-2:] + cols[:-2]
df_test = df_test[cols]

df_test.to_csv(os.path.join('results', 'probabilities_' + mname + '.csv')) # save temporarily, so rest can run on cheaper instance
df_test

,fname,name_split,0,1,2,3,4,5,6,7,...,56,57,58,59,60,61,62,63,64,65
0,0.wav,0_1.wav,0.000047,0.000059,0.000107,0.000193,0.001508,0.000048,0.000087,0.000023,...,0.000149,0.000138,0.000036,0.000090,0.000092,0.000797,0.000218,0.000121,0.000607,0.000333
1,1.wav,1_1.wav,0.000190,0.000234,0.000161,0.000068,0.000035,0.000052,0.004849,0.000300,...,0.000135,0.000018,0.000244,0.000110,0.981429,0.000121,0.000678,0.000103,0.000072,0.000073
2,3.wav,3_1.wav,0.008842,0.000765,0.002313,0.000570,0.001853,0.001901,0.000623,0.000632,...,0.000330,0.000651,0.004856,0.005736,0.002195,0.001411,0.003128,0.004884,0.000601,0.002155
3,3.wav,3_2.wav,0.004218,0.000722,0.001667,0.000386,0.001204,0.005065,0.000340,0.000724,...,0.000455,0.000827,0.005629,0.002390,0.000959,0.000990,0.002133,0.001974,0.000660,0.001002
4,3.wav,3_3.wav,0.005288,0.000394,0.000896,0.000606,0.001618,0.006221,0.000478,0.001568,...,0.000682,0.000552,0.004788,0.001240,0.001057,0.001090,0.001096,0.001161,0.000432,0.000370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1727,553.wav,553_3.wav,0.000299,0.000126,0.000213,0.000199,0.000270,0.003043,0.000530,0.000122,...,0.000159,0.000229,0.000455,0.000117,0.000173,0.000270,0.000244,0.000980,0.000279,0.000829
1728,553.wav,553_4.wav,0.000140,0.000151,0.000365,0.000306,0.000549,0.003737,0.000347,0.000167,...,0.000163,0.000183,0.000389,0.000083,0.000181,0.000247,0.000146,0.001166,0.000501,0.000531
1729,553.wav,553_5.wav,0.000324,0.000421,0.000977,0.000717,0.000654,0.003574,0.000213,0.000602,...,0.000266,0.000463,0.000450,0.000228,0.003722,0.000388,0.000306,0.003942,0.001202,0.001524
1730,555.wav,555_1.wav,0.000393,0.001367,0.000328,0.000142,0.001228,0.001066,0.000129,0.006057,...,0.002191,0.000162,0.001006,0.000948,0.003538,0.001340,0.000519,0.001058,0.000560,0.001464


# Get predictions

In [9]:
# Averaging over proabilities
df_test_avg = df_test.groupby(['fname'])[list(range(0,66,1))].mean().idxmax(axis='columns').reset_index()
#df.groupby(['original_filename', 'y_true'])[list(range(1,67,1))].mean()#.to_csv('tmp.csv')
df_test_avg.columns = ['file_name', 'predicted_class_id']

# Save 
df_test_avg.to_csv(os.path.join('results', 'predictions_' + mname + '.csv'), index=False) # save temporarily, so rest can run on cheaper instance
df_test_avg

,file_name,predicted_class_id
0,0.wav,14
1,1.wav,60
2,10.wav,26
3,100.wav,56
4,101.wav,57
...,...,...
551,95.wav,21
552,96.wav,43
553,97.wav,12
554,98.wav,40


In [3]:
import pandas as pd

In [5]:
df_test_now = pd.read_csv('results/predictions_gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed]@2023-07-16T12:59:29_epoch=19_epochs=[0, 20].csv')
df_test_9days_ago = pd.read_csv('results/oldRuns/gdsc22_passt_kd_explr_t40_f4_45epochs@2023-07-06T08:17:35_test_predictions_mean.csv')

In [6]:
pd.set_option("display.max_rows", None)
display(df_test_now.join(df_test_9days_ago, how='inner', lsuffix="_new", rsuffix="_old"))
pd.reset_option("^display")

,file_name_new,predicted_class_id_new,file_name_old,predicted_class_id_old
0,0.wav,14,0.wav,14
1,1.wav,60,1.wav,60
2,10.wav,26,10.wav,26
3,100.wav,56,100.wav,56
4,101.wav,57,101.wav,57
5,102.wav,10,102.wav,26
6,103.wav,5,103.wav,5
7,104.wav,29,104.wav,29
8,105.wav,12,105.wav,12
9,106.wav,38,106.wav,38


In [138]:
import pandas as pd

df_prob = pd.read_csv('./results/probabilities_gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed; rolling=0.5s; color_noise]@2023-07-16T10:14:41_epoch=19_epochs=[0, 20].csv')

In [139]:
idx = df_prob[df_prob.isna().any(axis=1)].index
df_prob = df_prob.drop(idx).drop(columns='Unnamed: 0').reset_index(drop=True)

In [128]:
df_prob

,fname,name_split,0,1,2,3,4,5,6,7,...,56,57,58,59,60,61,62,63,64,65
0,0.wav,0_1.wav,0.000029,0.000020,0.000050,0.000121,0.000533,0.000022,0.000047,0.000010,...,0.000182,0.000060,0.000019,0.000028,0.000023,0.000237,0.000105,0.000045,0.000240,0.000129
1,1.wav,1_1.wav,0.000080,0.000248,0.000082,0.000024,0.000015,0.000020,0.005490,0.000085,...,0.000120,0.000010,0.000058,0.000016,0.986402,0.000140,0.000183,0.000049,0.000050,0.000046
2,3.wav,3_1.wav,0.003711,0.000380,0.000890,0.000274,0.000633,0.000380,0.000197,0.000229,...,0.000270,0.000224,0.002016,0.002200,0.000536,0.001152,0.002464,0.001086,0.000244,0.001011
3,3.wav,3_2.wav,0.001966,0.000350,0.000642,0.000202,0.000771,0.001207,0.000138,0.000295,...,0.000558,0.000339,0.003169,0.000922,0.000309,0.000907,0.002145,0.001005,0.000200,0.000497
4,3.wav,3_3.wav,0.001752,0.000154,0.000268,0.000196,0.000446,0.000640,0.000103,0.000232,...,0.000320,0.000112,0.001952,0.000162,0.000236,0.000365,0.000588,0.000257,0.000092,0.000131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,553.wav,553_3.wav,0.000111,0.000104,0.000167,0.000130,0.000279,0.001115,0.000333,0.000145,...,0.000199,0.000167,0.000346,0.000096,0.000195,0.000127,0.000116,0.000561,0.000158,0.000379
1727,553.wav,553_4.wav,0.000034,0.000084,0.000099,0.000060,0.000240,0.000726,0.000129,0.000098,...,0.000067,0.000077,0.000094,0.000032,0.000089,0.000036,0.000046,0.000283,0.000102,0.000161
1728,553.wav,553_5.wav,0.000120,0.000337,0.000261,0.000124,0.000299,0.000793,0.000072,0.000189,...,0.000113,0.000377,0.000103,0.000102,0.001038,0.000057,0.000057,0.001572,0.000410,0.000661
1729,555.wav,555_1.wav,0.000117,0.000380,0.000079,0.000044,0.000301,0.000126,0.000020,0.005934,...,0.000837,0.000039,0.000313,0.000107,0.003380,0.000149,0.000128,0.000254,0.000195,0.000435


In [140]:
def return_maxidx_and_prob(x):
    max_column = x[2:].astype(float).idxmax()
    max_prob = x.loc[max_column]
    return pd.Series([max_column, max_prob], index=["vote", "prob"])

In [143]:
df_prob[["vote", "prob"]] = (df_prob.apply(return_maxidx_and_prob, axis=1, result_type="expand"))

In [144]:
df_prob

,fname,name_split,0,1,2,3,4,5,6,7,...,58,59,60,61,62,63,64,65,vote,prob
0,0.wav,0_1.wav,0.000029,0.000020,0.000050,0.000121,0.000533,0.000022,0.000047,0.000010,...,0.000019,0.000028,0.000023,0.000237,0.000105,0.000045,0.000240,0.000129,14,0.991256
1,1.wav,1_1.wav,0.000080,0.000248,0.000082,0.000024,0.000015,0.000020,0.005490,0.000085,...,0.000058,0.000016,0.986402,0.000140,0.000183,0.000049,0.000050,0.000046,60,0.986402
2,3.wav,3_1.wav,0.003711,0.000380,0.000890,0.000274,0.000633,0.000380,0.000197,0.000229,...,0.002016,0.002200,0.000536,0.001152,0.002464,0.001086,0.000244,0.001011,19,0.650720
3,3.wav,3_2.wav,0.001966,0.000350,0.000642,0.000202,0.000771,0.001207,0.000138,0.000295,...,0.003169,0.000922,0.000309,0.000907,0.002145,0.001005,0.000200,0.000497,19,0.591951
4,3.wav,3_3.wav,0.001752,0.000154,0.000268,0.000196,0.000446,0.000640,0.000103,0.000232,...,0.001952,0.000162,0.000236,0.000365,0.000588,0.000257,0.000092,0.000131,43,0.753500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,553.wav,553_3.wav,0.000111,0.000104,0.000167,0.000130,0.000279,0.001115,0.000333,0.000145,...,0.000346,0.000096,0.000195,0.000127,0.000116,0.000561,0.000158,0.000379,41,0.969945
1727,553.wav,553_4.wav,0.000034,0.000084,0.000099,0.000060,0.000240,0.000726,0.000129,0.000098,...,0.000094,0.000032,0.000089,0.000036,0.000046,0.000283,0.000102,0.000161,41,0.986092
1728,553.wav,553_5.wav,0.000120,0.000337,0.000261,0.000124,0.000299,0.000793,0.000072,0.000189,...,0.000103,0.000102,0.001038,0.000057,0.000057,0.001572,0.000410,0.000661,41,0.906646
1729,555.wav,555_1.wav,0.000117,0.000380,0.000079,0.000044,0.000301,0.000126,0.000020,0.005934,...,0.000313,0.000107,0.003380,0.000149,0.000128,0.000254,0.000195,0.000435,10,0.920216


In [182]:
# def get_majority_vote(x):
#     # fallback to probability voting
#     counts = x['vote'].value_counts()
#     if len(counts) == 2:
#         print(counts.idxmax())
#         print(x)        
#         print(counts)
        #print(index_max_prob, x)
        
#         if len(x) == 2:
#         index_max_prob = x['prob'].idxmax()
    #print(counts)
    #print(counts.idxmax())

In [203]:
def get_majority_vote(x):
    vote_mode = x['vote'].mode()
    result = vote_mode[0]    
    if len(vote_mode) > 1:
        # don't even want to imagine if there are 3 values...
        val1 = vote_mode[0]
        val2 = vote_mode[1]
        
        #fallback to one probability
        idx1 = (x[x['vote'] == val1].index)
        val1_maxprob = x.loc[idx1, 'prob'].max()
        
        idx2 = (x[x['vote'] == val2].index)
        val2_maxprob = x.loc[idx2, 'prob'].max()
        
        if val1_maxprob >= val2_maxprob:
            result = val1
        else:
            result = val2
        
    return pd.Series([result], index=['predicted_class_id'])
        


In [208]:
pd.set_option("display.max_rows", None)
#display(df_prob.loc[:, ['fname', 'vote']])
pd.reset_option("^display")
df_prob_pred = df_prob.loc[:, ['fname', 'vote', 'prob']].groupby('fname', as_index=False).apply(get_majority_vote)

In [210]:
df_prob_pred_gsk = pd.read_csv('./results/predictions_gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_mean_loss_train+val_stratified_epochs20_transformation=[normed; rolling=0.5s; color_noise]@2023-07-16T10:14:41_epoch=19_epochs=[0, 20].csv')

In [214]:
pd.set_option("display.max_rows", None)
display(df_prob_pred.join(df_prob_pred_gsk, how='inner', lsuffix="da", rsuffix="gsk"))
pd.reset_option("^display")

,fname,predicted_class_idda,file_name,predicted_class_idgsk
0,0.wav,14,0.wav,14
1,1.wav,60,1.wav,60
2,10.wav,26,10.wav,26
3,100.wav,56,100.wav,56
4,101.wav,57,101.wav,57
5,102.wav,26,102.wav,26
6,103.wav,5,103.wav,5
7,104.wav,29,104.wav,29
8,105.wav,12,105.wav,12
9,106.wav,38,106.wav,38


In [211]:
df_prob_pred_gsk

,file_name,predicted_class_id
0,0.wav,14
1,1.wav,60
2,10.wav,26
3,100.wav,56
4,101.wav,57
...,...,...
551,95.wav,21
552,96.wav,43
553,97.wav,12
554,98.wav,19


# Legacy cells - Discard after rest is running

### Define dataloaders

## Load the model

# Run predictions on validation
Remember to remobe cnt, break, and .cuda() when running the real model

# Gather predictions to a dataframe

### Skipping F1 calculations right now, as I don't have the results. 
but this should be the basis

That's a lie... now taking a look:

## See if we can run predictions on test dataset

### Gather results to a csv

# Taking a look at class imbalances?

# Check if hierachical makes any sense
TL;Dr: No it doesn't

## Check for order

## Check for Suborder

## Check for Family